## Creating Audio File

Checkout https://beta.elevenlabs.io/history for full log of audio creation

In [1]:
file_name = "./play_kitten"
mp3_file_name = file_name + ".mp3"
wav_file_name = file_name + ".wav"

generate_new_audio = False
name_of_voice = "Bella"
text = "I love you"

In [2]:
# Get voice ID
import requests

r1 = requests.get('https://api.elevenlabs.io/v1/voices')
resp = r1.json()
rid = ""
for voice in resp["voices"]:
    if voice["name"] == name_of_voice:
        print(voice)
        rid = voice["voice_id"]
        break;
rid

{'voice_id': 'EXAVITQu4vr4xnSDxMaL', 'name': 'Bella', 'samples': None, 'category': 'premade', 'fine_tuning': {'is_allowed_to_fine_tune': False, 'fine_tuning_requested': False, 'finetuning_state': 'not_started', 'verification_attempts': None, 'verification_attempts_count': 0}, 'labels': {}, 'preview_url': 'https://storage.googleapis.com/eleven-public-prod/premade/voices/EXAVITQu4vr4xnSDxMaL/04365bce-98cc-4e99-9f10-56b60680cda9.mp3', 'available_for_tiers': [], 'settings': None}


'EXAVITQu4vr4xnSDxMaL'

In [3]:
# Get audio file, minimize runs cost money
import json

if generate_new_audio:
    url = "https://api.elevenlabs.io/v1/text-to-speech/{voice_id}".format(voice_id=rid)
    print(url)
    headers = {'xi-api-key': "aa42307ce6bfbefcfd2abd4d8634127b"}
    payload = {
      "text": text,
      "voice_settings": {
        "stability": 0.3,
        "similarity_boost": 0.75
      }
    }

    r2 = requests.post(url, data=json.dumps(payload), headers=headers)
    print(r2)

In [4]:
# Save and play audio file
import os

if generate_new_audio:
    with open(mp3_file_name, 'wb') as f:
        f.write(r2.content)

    os.system("mpg123 " + mp3_file_name)

In [5]:
# convert mp3 to wav                                                            
from pydub import AudioSegment
from pydub.playback import play

if generate_new_audio:
    sound = AudioSegment.from_mp3(mp3_file_name)
    sound.export(wav_file_name, format="wav")

    # test sound
    song = AudioSegment.from_wav(wav_file_name)
    play(song)

## Generate phonemes
- Uses https://github.com/m-bain/whisperX

In [6]:
# no extension, must be wav
import os
cwd = os.getcwd() #pwd
print(cwd)

audio_file_name = wav_file_name
base_audio_file_name = os.path.splitext(os.path.basename(audio_file_name))[0]
print("File name:", audio_file_name)
print("Base name:", base_audio_file_name)

/Users/jfuentes/Projects/infinitenews/AAAI22-one-shot-talking-face/scripts
File name: ./play_kitten.wav
Base name: play_kitten


In [7]:
import whisperx

device = "cpu" 
# transcribe with original whisper
model = whisperx.load_model("large", device)

/Users/jfuentes/Projects/infinitenews/AAAI22-one-shot-talking-face/venv/lib/python3.9/site-packages/torchaudio/_internal/module_utils.py:99: UserWarning: Failed to import soundfile. 'soundfile' backend is not available.
  warnings.warn("Failed to import soundfile. 'soundfile' backend is not available.")


In [8]:
import time

start = time.time()

result = model.transcribe(audio_file_name)

print(result["segments"]) # before alignment

# load alignment model and metadata
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)

# align whisper output
result_aligned = whisperx.align(result["segments"], model_a, metadata, audio_file_name, device)

# print(result_aligned["segments"]) # after alignment
end = time.time()
print("Time", end - start)

result_aligned["word_segments"]

/Users/jfuentes/Projects/infinitenews/AAAI22-one-shot-talking-face/venv/lib/python3.9/site-packages/whisperx/transcribe.py:83: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


RuntimeError: Failed to load audio: ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzm

In [ ]:
test_words = ["that's", "Help! ", "multiverse", "cat", "permit", "surprise", ",whales"]

In [ ]:
import nltk

try:
    arpabet = nltk.corpus.cmudict.dict()
except LookupError:
    nltk.download('cmudict')
    arpabet = nltk.corpus.cmudict.dict()

In [ ]:
import re
from functools import lru_cache
from itertools import product as iterprod

def cleanWord(word):
    regex = r"[^\w\']"
    return re.sub(regex, "", word).lower()

def arpabetWithoutNumbers(word):
    list_of_phones = []
    for phones in arpabet[word]:
        new_phones = ["".join(filter(lambda x: x.isalpha(), phone)) for phone in phones]
        list_of_phones.append(new_phones)
    return list_of_phones

def wordbreak(s):
    s = cleanWord(s)
    if s in arpabet:
        return arpabetWithoutNumbers(s)
    middle = len(s)/2
    partition = sorted(list(range(len(s))), key=lambda x: (x-middle)**2-x)
    for i in partition:
        pre, suf = (s[:i], s[i:])
        if pre in arpabet and wordbreak(suf) is not None:
            return [x+y for x,y in iterprod(arpabetWithoutNumbers(pre), wordbreak(suf))]
    return None

# Example words: 
for w in test_words:
    print(wordbreak(w))

In [ ]:
def letterToPhone(word, phones):
    # print("LTP", word, phones)
    if word == "":
        return []

    for i in reversed(range(1, len(word))):
        for j in range(0, len(word) - i + 1):
            subword = word[j:i+j]
            cur_possible_phones = wordbreak(subword)
            #For when has contractions
            if not cur_possible_phones:
                print("Skipping", i, j, subword, cur_possible_phones)
                continue
            else:
                cur_possible_phones = cur_possible_phones + [[subword.upper()]]
                print(i, j, subword, cur_possible_phones)

            for cur_phones in cur_possible_phones:
                extra_length = len(phones) - len(cur_phones)
#                 print("EL", extra_length)
                if extra_length > 0:
                    for a in range(0, extra_length + 1):
                        subphones_end = len(phones) - (extra_length - a)
                        subphones = phones[a:subphones_end]
                        if len(subphones) != len(cur_phones):
                            print("Lengths not equal....")
                        #print(subphones, cur_phones)
                        if subphones == cur_phones:
                            #print("MATCH", subword, subphones)
                            first = letterToPhone(word[0:j], phones[0:a])
                            cur = letterToPhone(subword, subphones)
                            last = letterToPhone(word[i+j:], phones[subphones_end:])
                            # print("LTP Complete", first, cur, last)
                            return first + cur + last
                    
    return [[word, phones]]

for segment in result_aligned["word_segments"]:
    word = segment["text"]

# for word in test_words:
    print(word)
    print(letterToPhone(word, wordbreak(word)[0]))
    print([word, wordbreak(word)[0]], "\n")

In [ ]:
# {'text': 'Good', 'start': 0.10120481927710842, 'end': 0.32385542168674697},
# INTO
# [
#     {
#         "word": "sil",
#         "phones": [
#             {
#                 "ph": "SIL",
#                 "bg": 0,
#                 "ed": 126
#             }
#         ]
#     },
#     {
#         "word": "YOU",
#         "phones": [
#             {
#                 "ph": "Y",
#                 "bg": 126,
#                 "ed": 138
#             },
#             {
#                 "ph": "UW",
#                 "bg": 138,
#                 "ed": 141
#             }
#         ]
#     },
from math import trunc

def makeSegment(word, phones, start, end):
    total = end - start
    time_per_phone = trunc(total / len(phones))
#     print(total, time_per_phone)
    expanded_phones = [{"ph": phone, "bg": start + (i * time_per_phone), "ed": start + ((i + 1) * time_per_phone)} for i, phone in enumerate(phones)]
    return {"word": word, "phones": expanded_phones}

def makeSil(start, end):
    return {"word": "sil", "phones": [{"ph": "SIL", "bg": start, "ed": end}]}
  
all_phonemes = []
for segment in result_aligned["word_segments"]:
    word = segment["text"]
    #Get rid of numbers in phones
    phonemes = wordbreak(word)[0]
    print(phonemes)
    start_time = trunc(segment["start"] * 100)
    end_time = trunc(segment["end"] * 100)
    
    if len(all_phonemes) > 0:
        last = all_phonemes[-1]
        last_end = last["phones"][-1]["ed"]
        if start_time - last_end > 5:
            sil = makeSil(last_end, start_time)
            all_phonemes.append(sil)
        else:
            start_time = last_end
    else:
        sil = makeSil(0, start_time)
        all_phonemes.append(sil)
        
    segment = makeSegment(word, phonemes, start_time, end_time)
    all_phonemes.append(segment)

all_phonemes

In [63]:
import json

file_name = "../samples/phonemes/{name}.json".format(name=base_audio_file_name)
print("Writing to ", file_name)
with open(file_name, 'w') as f1:
    json.dump(all_phonemes, f1)

Writing to  ../samples/phonemes/kitten.json


In [55]:
all_phonemes


[{'word': 'sil', 'phones': [{'ph': 'SIL', 'bg': 0, 'ed': 10}]},
 {'word': 'I', 'phones': [{'ph': 'AY', 'bg': 10, 'ed': 16}]},
 {'word': 'love',
  'phones': [{'ph': 'L', 'bg': 16, 'ed': 23},
   {'ph': 'AH', 'bg': 23, 'ed': 30},
   {'ph': 'V', 'bg': 30, 'ed': 37}]},
 {'word': 'you.',
  'phones': [{'ph': 'Y', 'bg': 37, 'ed': 43},
   {'ph': 'UW', 'bg': 43, 'ed': 49}]}]

http://localhost:1337/api/modules/1?populate=Image1


{'data': {'id': 1,
  'attributes': {'Text1': "Good evening everyone and welcome to this special report. As you know, we’ve been tracking a strange uptick in activity in the multiverse - and as we feared, it appears that space whales have been invading our universe through a rift in the multiverse. \n\nThese massive creatures, which can range in size from the size of a small ship to larger than a mountain, are wreaking havoc on the balance of the multiverse. What's worse is that they seem to be coming from a totally different dimension, one that our scientists didn't even know existed until now.\n\nWe’ve seen some strange things in the multiverse before, but never anything as awe-inspiring and destructive as these space whales. They’ve already destroyed several planets and star systems, and it looks like they’re only getting started. \n\nThe multiverse is in a state of crisis, but thankfully our scientists are working around the clock to try and close the rift and prevent any more of th

In [65]:
import torch
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

MPS not available because the current MacOS version is not 12.3+ and/or you do not have an MPS-enabled device on this machine.
